In [1]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.10.1


In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
        image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
                                        target_size=(224, 224))
        image_arr = tf.keras.utils.img_to_array(image)
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

((407, 224, 224, 3), (407,))

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((305, 224, 224, 3), (102, 224, 224, 3), (305,), (102,))

In [5]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_train.shape, y_test.shape

((305, 3), (102, 3))

In [6]:
from tensorflow.keras import layers, models

# Define the CNN architecture
def create_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [7]:
# Create the CNN model
input_shape = X_train.shape[1:]
num_classes = 3
model = create_cnn(input_shape, num_classes)

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [10]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, 
                    batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
5/5 [==============================] - 37s 7s/step - loss: 9.4076 - accuracy: 0.3443 - val_loss: 3.1020 - val_accuracy: 0.4902
Epoch 2/10
5/5 [==============================] - 33s 7s/step - loss: 1.4434 - accuracy: 0.5410 - val_loss: 0.9361 - val_accuracy: 0.5686
Epoch 3/10
5/5 [==============================] - 36s 7s/step - loss: 0.8198 - accuracy: 0.6361 - val_loss: 0.7668 - val_accuracy: 0.5784
Epoch 4/10
5/5 [==============================] - 35s 7s/step - loss: 0.6055 - accuracy: 0.7541 - val_loss: 0.7192 - val_accuracy: 0.6373
Epoch 5/10
5/5 [==============================] - 34s 7s/step - loss: 0.5433 - accuracy: 0.7344 - val_loss: 0.9307 - val_accuracy: 0.5980
Epoch 6/10
5/5 [==============================] - 34s 7s/step - loss: 0.5340 - accuracy: 0.7902 - val_loss: 0.4647 - val_accuracy: 0.7647
Epoch 7/10
5/5 [==============================] - 36s 7s/step - loss: 0.3502 - accuracy: 0.8590 - val_loss: 0.5170 - val_accuracy: 0.7451
Epoch 8/10
5/5 [==================

In [11]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

4/4 [==============================] - 3s 758ms/step - loss: 0.5340 - accuracy: 0.7843
Test loss: 0.5340490341186523
Test accuracy: 0.7843137383460999


In [5]:
# import tensorflow as tf
# from tensorflow.keras.applications.densenet import DenseNet121

# # Load the pre-trained DenseNet121 model
# model = DenseNet121(weights='imagenet')

In [6]:
# # Create a string variable
# summary_string = model.summary()

# # Save the string variable to a file
# with open("imagenet_model.txt", "w") as f:
#     f.write(summary_string)

# f.close()
# f = open("imagenet_model.txt", "w")
# f.write(summary_string)
# f.close()

In [5]:
import tensorflow.keras as K

# Define convolutional block
def conv_block(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):
    x = K.layers.Conv2D(filters, kernel_size, strides=strides, padding=padding, use_bias=False)(x)
    x = K.layers.BatchNormalization()(x)
    x = K.layers.ReLU()(x)
    return x

# Define dense block
def dense_block(x, filters, repetition):
    for _ in range(repetition):
        y = conv_block(x, 4 * filters)
        y = conv_block(y, filters)
        x = K.layers.concatenate([x, y])
    return x

# Define transition block
def transition_block(x, filters):
    x = conv_block(x, filters)
    x = K.layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

# Define DenseNet121 model
def DenseNet121(input_shape=(224, 224, 3), num_classes=3):
    inputs = K.layers.Input(input_shape)

    # Initial convolution
    x = conv_block(inputs, 64)

    # Dense blocks
    x = dense_block(x, 64, 6)
    x = transition_block(x, 128)
    x = dense_block(x, 128, 12)
    x = transition_block(x, 256)
    x = dense_block(x, 256, 24)
    x = transition_block(x, 512)
    x = dense_block(x, 512, 16)

    # Global average pooling
    x = K.layers.GlobalAveragePooling2D()(x)

    # Output layer
    outputs = K.layers.Dense(num_classes, activation='softmax')(x)

    return K.Model(inputs=inputs, outputs=outputs)

In [6]:
# Create the model
model = DenseNet121()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  1728        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

C:\Users\Mukesh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\layer_utils.py:124: RuntimeWarning: overflow encountered in scalar add
  return int(sum(np.prod(p) for p in standardized_weight_shapes))


In [93]:
test_arr = tf.expand_dims(image_arr, axis=0)
test_arr.shape

TensorShape([1, 224, 224, 3])

In [94]:
# Make a prediction
predictions = model.predict(test_arr)

1/1 [==============================] - 0s 334ms/step


In [72]:
# Get the predicted class
predicted_class = tf.argmax(predictions[0])

In [73]:
# Print the predicted class
print('Predicted class:', predicted_class)

Predicted class: tf.Tensor(701, shape=(), dtype=int64)


In [74]:
# predicted_class

In [75]:
predicted_class

<tf.Tensor: shape=(), dtype=int64, numpy=701>

In [76]:
np.argmax(predictions[0])

701

In [86]:
(predictions[0][701])*100

66.50622487068176

In [96]:
top_5preds = tf.keras.applications.densenet.decode_predictions(
    predictions, top=5
)
top_5preds

[[('n03888257', 'parachute', 0.66506225),
  ('n02782093', 'balloon', 0.3104138),
  ('n03944341', 'pinwheel', 0.024471207),
  ('n02701002', 'ambulance', 5.167244e-05),
  ('n06359193', 'web_site', 7.0670364e-07)]]

In [97]:
type(top_5preds)

list

In [112]:
top_5preds[0][0][1]

'parachute'